<a href="https://colab.research.google.com/github/annasuranyi/NLP_Projekt/blob/main/NLP_Projekt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Szöveg alapú ajánlórendszer**

#1. Könyvtárak telepítése

In [ ]:
pip install requests pandas scikit-learn

#2. Projekt kódja

## Adatok létrehozása - TMDB API kulcs megadása

In [ ]:
import requests
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

API_KEY = "6b5bdc65c4ca681df7bbe2158ba996ee"  # Cseréld le a saját TMDB API kulcsodra
BASE_URL = "https://api.themoviedb.org/3"

def fetch_movies(api_key, num_movies=100):
    """
    Lekéri a TMDB API-ból az aktuálisan népszerű filmeket több oldalon keresztül.

    Paraméterek:
        api_key (str): A TMDB API kulcs.
        num_movies (int): A lekérendő filmek száma.

    Return:
        DataFrame: A filmek címét, leírását és műfajait tartalmazó DataFrame.
    """
    movies = []
    page = 1
    while len(movies) < num_movies:
        # Népszerű filmek lekérdezése oldalanként
        url = f"{BASE_URL}/movie/popular"
        params = {"api_key": api_key, "language": "en-US", "page": page}
        response = requests.get(url, params=params)

        if response.status_code != 200:
            raise Exception(f"API hiba: {response.status_code}, {response.text}")

        data = response.json()
        for movie in data["results"]:
            if len(movies) >= num_movies:
                break
            title = movie["title"]
            description = movie.get("overview", "Nincs leírás")
            genres = fetch_genres(movie["genre_ids"], api_key)
            movies.append({"title": title, "description": description, "genres": genres})

        page += 1  # Következő oldal betöltése

    return pd.DataFrame(movies)

def fetch_genres(genre_ids, api_key):
    """
    Lekéri a műfajok nevét az azonosítók alapján.

    Paraméterek:
        genre_ids (list): A műfajok azonosítói.
        api_key (str): A TMDB API kulcs.

    Return:
        str: A műfajok neve vesszővel elválasztva.
    """
    # Műfajok listája (csak egyszer kellene letölteni, itt dinamikusan van megoldva)
    genre_url = f"{BASE_URL}/genre/movie/list"
    params = {"api_key": api_key, "language": "en-US"}
    response = requests.get(genre_url, params=params)

    if response.status_code != 200:
        raise Exception(f"API hiba a műfajok lekérdezésekor: {response.status_code}, {response.text}")

    genres_data = response.json()["genres"]
    genres_dict = {genre["id"]: genre["name"] for genre in genres_data}
    return ", ".join([genres_dict[genre_id] for genre_id in genre_ids if genre_id in genres_dict])

## Adatok letöltése

In [ ]:
print("Adatok letöltése a TMDB API-ból...")
df = fetch_movies(API_KEY, num_movies=200)
print("Sikeresen letöltött adatok:")
print(df.head())

## TF-IDF vektorok létrehozása - A TF-IDF vektorok létrehozásához a TfidfVectorizer-t használjuk.

In [ ]:
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(df['description'])

## Koszinusz hasonlóság kiszámítása - A koszinusz hasonlóság mátrixot a cosine_similarity függvénnyel számoljuk ki.

In [ ]:
# A koszinusz hasonlóság mátrix számítása a TF-IDF vektorok alapján
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

## Ajánló funkció - Egy egyszerű ajánló funkció (recommend_movies) kiszámítja a legjobban hasonlító filmeket.

In [ ]:
def recommend_movies(title, cosine_sim=cosine_sim):
    """
    Visszaadja a megadott filmhez legjobban hasonlító filmek címét.

    Paraméterek:
        title (str): A film címe, amelyhez hasonlót keresünk.
        cosine_sim (ndarray): A koszinusz hasonlóság mátrix.

    Return:
        list: Az 5 legjobban hasonlító film címe.
    """
    if title not in df['title'].values:
        return ["Nincs ilyen film az adatbázisban."]

    # A film indexének megkeresése a DataFrame-ben
    idx = df[df['title'] == title].index[0]

    # Hasonlósági pontszámok lekérése a kiválasztott filmhez
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Rendezés a hasonlóság szerint, csökkenő sorrendben
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # A top 5 legjobban hasonlító film indexei (kivéve saját maga)
    sim_scores = sim_scores[1:6]
    movie_indices = [i[0] for i in sim_scores]

    # Visszatérünk a filmek címeivel
    return df['title'].iloc[movie_indices]

## Felhasználói interakció - A felhasználó megadhat egy filmcímet, és a program visszaadja az 5 leginkább hasonló filmet.

In [ ]:
if __name__ == "__main__":
    print("\nÜdvözöllek a film ajánló rendszerben!")
    print("Írd be egy film címét, és ajánlok hasonló filmeket.")
    print("Elérhető filmek:")
    print(df['title'].tolist())
    print("\n")

    # Felhasználói bemenet
    user_movie = input("Adj meg egy filmcímet: ")

    # Ajánlások
    recommendations = recommend_movies(user_movie)
    print(f"\nAjánlott filmek az '{user_movie}' című filmhez:")
    for i, movie in enumerate(recommendations, 1):
        print(f"{i}. {movie}")

#3. Statisztika

## Filmek száma

In [ ]:
print(f"Film adatbázis mérete: {len(df)} film")

Film adatbázis mérete: 200 film


## Leírások hosszának eloszlása

In [ ]:
df['description_length'] = df['description'].apply(len)
print(df['description_length'].describe())

# Histogram a hosszok eloszlásáról
import matplotlib.pyplot as plt
df['description_length'].hist(bins=20)
plt.title("Filmleírások hosszának eloszlása")
plt.xlabel("Hossz (karakterek száma)")
plt.ylabel("Gyakoriság")
plt.show()

## Melyek a leggyakoribb műfajok az adathalmazban?

In [ ]:
from collections import Counter

genre_counts = Counter(", ".join(df['genres']).split(", "))
print("Műfajok gyakorisága:")
for genre, count in genre_counts.most_common():
    print(f"{genre}: {count}")

# Műfajok megjelenítése oszlopdiagramon
plt.bar(genre_counts.keys(), genre_counts.values())
plt.title("Műfajok gyakorisága")
plt.xlabel("Műfajok")
plt.ylabel("Előfordulások száma")
plt.xticks(rotation=45)
plt.show()

## Átlagos hasonlóság az ajánlásokban

In [ ]:
def average_similarity(title, cosine_sim=cosine_sim):
    idx = df[df['title'] == title].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]  # Top 5 ajánlás
    avg_sim = sum([score[1] for score in sim_scores]) / len(sim_scores)
    return avg_sim

df['average_similarity'] = df['title'].apply(average_similarity)
print("Átlagos hasonlóság az ajánlások között:")
print(df[['title', 'average_similarity']])

## Hasonlósági mátrix vizualizációja

In [ ]:
# prompt: 20 random filmhez készíts hasonlósági mátrix vizualizációt úgy, hogy látszódjanak a filmcímek, a hasonlósági mátrix a műfajokra vonatkozzon

import matplotlib.pyplot as plt
import seaborn as sns

# Select the first 20 movies
df_20 = df.head(20)

# Create a TF-IDF vectorizer for genres
genre_vectorizer = TfidfVectorizer()
genre_tfidf_matrix = genre_vectorizer.fit_transform(df_20['genres'])

# Calculate cosine similarity based on genres
genre_cosine_sim = cosine_similarity(genre_tfidf_matrix, genre_tfidf_matrix)

# Create the visualization
plt.figure(figsize=(10, 8))
sns.heatmap(genre_cosine_sim, annot=False, cmap="viridis", xticklabels=df_20['title'], yticklabels=df_20['title'])
plt.title("Genre Similarity Matrix for 20 Movies")
plt.xlabel("Movie Titles")
plt.ylabel("Movie Titles")
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.show()

## Filmek UMAP vetítése (TF-IDF mátrix alapján)

In [ ]:
pip install umap-learn matplotlib plotly

In [ ]:
import umap
import matplotlib.pyplot as plt
import seaborn as sns

# UMAP model létrehozása és dimenziócsökkentés 2D-re
umap_model = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2, random_state=42)
umap_embedding = umap_model.fit_transform(tfidf_matrix.toarray())

# 2D scatter plot a filmekhez
plt.figure(figsize=(12, 8))
sns.scatterplot(
    x=umap_embedding[:, 0],
    y=umap_embedding[:, 1],
    hue=df['genres'],
    palette="viridis",
    legend='full'
)
plt.title("Filmek UMAP vetítése (TF-IDF mátrix alapján)", fontsize=16)
plt.xlabel("UMAP 1. komponens")
plt.ylabel("UMAP 2. komponens")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

In [ ]:
# prompt: Írj egy olyan kódot, ami  megjeleníti a 10 legnépszerűbb műfajt wordclouddal

from wordcloud import WordCloud
import matplotlib.pyplot as plt

# A genre_counts változó feltételezve van a korábbi kódból
# Ha nincs definiálva, akkor definiáld a korábbi kódban lévő módon

# Készíts egy WordCloud objektumot
wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(genre_counts)

# Megjelenítés
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title("10 Legnépszerűbb műfaj")
plt.show()